In [2]:
import numpy as np
import tensorflow as tf
from create_sentiment_featuresets import create_feature_sets_and_labels     

[nltk_data] Downloading package punkt to /home/suraj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/suraj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
train_x,train_y,test_x,test_y = create_feature_sets_and_labels('pos.txt','neg.txt')

423


In [4]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 2                         #
batch_size = 100

x = tf.placeholder('float',[None, len(train_x[0])])
y = tf.placeholder('float')

In [12]:
def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),   #
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                     'biases':tf.Variable(tf.random_normal([n_classes]))}
    # (input_data * weights) + biases
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']),hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']

    return output
    

In [13]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            
            i = 0                                                                       #
            while i < len(train_x):
                start = i
                end = i+batch_size
                
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
                _,c = sess.run([optimizer, cost], feed_dict = {x: batch_x,y: batch_y})
                epoch_loss += c
                i += batch_size
                
            print('Epoch',epoch+1, 'completed out of', hm_epochs,'loss:',epoch_loss)
         
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))                           #
        
train_neural_network(x)        
        
    
        

W0823 13:11:01.937619 140046543316800 deprecation.py:323] From <ipython-input-13-5448ca7312c7>:3: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W0823 13:11:07.895732 140046543316800 deprecation.py:323] From /home/suraj/anaconda3/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Epoch 1 completed out of 10 loss: 252569.8360595703
Epoch 2 completed out of 10 loss: 121988.57989501953
Epoch 3 completed out of 10 loss: 75553.19815063477
Epoch 4 completed out of 10 loss: 49550.51126098633
Epoch 5 completed out of 10 loss: 36186.4709777832
Epoch 6 completed out of 10 loss: 26982.584701538086
Epoch 7 completed out of 10 loss: 32798.47426223755
Epoch 8 completed out of 10 loss: 33300.0354347229
Epoch 9 completed out of 10 loss: 26308.996418952942
Epoch 10 completed out of 10 loss: 14949.436321258545
Accuracy: 0.5891182
